In [1]:
import pandas as pd
import numpy as np
from scipy import stats, integrate
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
from matplotlib.font_manager import FontProperties
myfont = FontProperties(fname = "C:\Windows\Fonts\simhei.ttf", size = 14)
sns.set(font = myfont.get_name())

In [2]:
cleanedDataPcf = pd.read_csv("CleanedData/pacifier.csv", index_col = 0, parse_dates=['review_date'])

encodedDataPcf = deepcopy(cleanedDataPcf)

cleanedDataMcw = pd.read_csv("CleanedData/microwave.csv", index_col = 0, parse_dates=['review_date'])

encodedDataMcw = deepcopy(cleanedDataMcw)

cleanedDataHdr = pd.read_csv("CleanedData/hair_dryer.csv", index_col = 0, parse_dates=['review_date'])

encodedDataHdr = deepcopy(cleanedDataHdr)

In [3]:
encodedDataPcf = encodedDataPcf[["customer_id", "review_id", 
                                 "product_id", "star_rating", 
                                 "helpful_votes", "total_votes", 
                                 "vine", "verified_purchase", 
                                 "review_headline", "review_body", 
                                "review_date"]]

encodedDataMcw = encodedDataMcw[["customer_id", "review_id", 
                                 "product_id", "star_rating", 
                                 "helpful_votes", "total_votes", 
                                 "vine", "verified_purchase", 
                                 "review_headline", "review_body", 
                                "review_date"]]

encodedDataHdr = encodedDataHdr[["customer_id", "review_id", 
                                 "product_id", "star_rating", 
                                 "helpful_votes", "total_votes", 
                                 "vine", "verified_purchase", 
                                 "review_headline", "review_body", 
                                "review_date"]]

只提取有意义的特征

In [4]:
encodedDataPcf["vine"] = (encodedDataPcf["vine"] == 'Y') | (encodedDataPcf["vine"] == 'y')

encodedDataMcw["vine"] = (encodedDataMcw["vine"] == 'Y') | (encodedDataMcw["vine"] == 'y')

encodedDataHdr["vine"] = (encodedDataHdr["vine"] == 'Y') | (encodedDataHdr["vine"] == 'y')

encodedDataPcf["verified_purchase"] = (encodedDataPcf["verified_purchase"] == 'Y') | (encodedDataPcf["verified_purchase"] == 'y')

encodedDataMcw["verified_purchase"] = (encodedDataMcw["verified_purchase"] == 'Y') | (encodedDataMcw["verified_purchase"] == 'y')

encodedDataHdr["verified_purchase"] = (encodedDataHdr["verified_purchase"] == 'Y') | (encodedDataHdr["verified_purchase"] == 'y')

把逻辑特征转为布尔值

In [5]:
encodedDataPcf["product_id"] = encodedDataPcf["product_id"].str.upper()

encodedDataMcw["product_id"] = encodedDataMcw["product_id"].str.upper()

encodedDataHdr["product_id"] = encodedDataHdr["product_id"].str.upper()

encodedDataPcf["customer_id"] = encodedDataPcf["customer_id"].astype("str")

encodedDataMcw["customer_id"] = encodedDataMcw["customer_id"].astype("str")

encodedDataHdr["customer_id"] = encodedDataHdr["customer_id"].astype("str")

encodedDataPcf["review_id"] = encodedDataPcf["review_id"].str.upper()

encodedDataMcw["review_id"] = encodedDataMcw["review_id"].str.upper()

encodedDataHdr["review_id"] = encodedDataHdr["review_id"].str.upper()

处理Unique Id

In [6]:
customerDataBase = {
    
}
"""
helpfulnessRatio [helpful, total]
isVine [bool]
purchasedRatio [purchased, total]
"""

for id in encodedDataPcf["customer_id"] :
    
    if(id not in customerDataBase) : customerDataBase[id] = {"helpfulnessRatio" : [1, 2], "isVine" : False, "purchasedRatio" : [0, 0]}
    
for id in encodedDataMcw["customer_id"] :
    
    if(id not in customerDataBase) : customerDataBase[id] = {"helpfulnessRatio" : [1, 2], "isVine" : False, "purchasedRatio" : [0, 0]}
    
for id in encodedDataHdr["customer_id"] :
    
    if(id not in customerDataBase) : customerDataBase[id] = {"helpfulnessRatio" : [1, 2], "isVine" : False, "purchasedRatio" : [0, 0]}

        
for i in range(0, len(encodedDataPcf)) :
    
    row = encodedDataPcf.iloc[i]
    
    customerDataBase[row["customer_id"]]["helpfulnessRatio"][0] += row["helpful_votes"]
    
    customerDataBase[row["customer_id"]]["helpfulnessRatio"][1] += row["total_votes"]
    
    customerDataBase[row["customer_id"]]["isVine"] |= row["vine"]
    
    customerDataBase[row["customer_id"]]["purchasedRatio"][0] += row["verified_purchase"]
    
    customerDataBase[row["customer_id"]]["purchasedRatio"][1] += 1
    
for i in range(0, len(encodedDataMcw)) :
    
    row = encodedDataMcw.iloc[i]
    
    customerDataBase[row["customer_id"]]["helpfulnessRatio"][0] += row["helpful_votes"]
    
    customerDataBase[row["customer_id"]]["helpfulnessRatio"][1] += row["total_votes"]
    
    customerDataBase[row["customer_id"]]["isVine"] |= row["vine"]
    
    customerDataBase[row["customer_id"]]["purchasedRatio"][0] += row["verified_purchase"]
    
    customerDataBase[row["customer_id"]]["purchasedRatio"][1] += 1
    

for i in range(0, len(encodedDataHdr)) :
    
    row = encodedDataHdr.iloc[i]
    
    customerDataBase[row["customer_id"]]["helpfulnessRatio"][0] += row["helpful_votes"]
    
    customerDataBase[row["customer_id"]]["helpfulnessRatio"][1] += row["total_votes"]
    
    customerDataBase[row["customer_id"]]["isVine"] |= row["vine"]
    
    customerDataBase[row["customer_id"]]["purchasedRatio"][0] += row["verified_purchase"]
    
    customerDataBase[row["customer_id"]]["purchasedRatio"][1] += 1
    

生成评论者数据库

In [7]:
customerDataBase

{'40626522': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '16290022': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '10216509': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '114040': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [2, 2]},
 '27971579': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '36369192': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '18742061': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '15312194': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '44791277': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '2943530': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '32717440': {'helpfulnessRatio': [1, 2],
  'isVine': False,
  'purchasedRatio': [1, 1]},
 '2761934': {